In [4]:
!pip install salib
!pip install seaborn

In [37]:
%display typeset

In [38]:
from SALib.sample import saltelli
from SALib.analyze import sobol
import seaborn as sns
import numpy as np
import pandas as pd
import pylab as plt
from multiprocessing import Pool

In [39]:
def ODE (t,y,params):
    S,E,I,A,R,D = y
    lambda_, alpha,p,theta,gamma, mu,mu1,mu2,t1,t2 = params

    # transição de mu1 para mu2 
    mu = ( ( (np.tanh(-t + t2) + 1 )/2 ) * abs(mu1 - mu2) ) + mu2
    
    # período de isolamento com queda de lambda_ com início em t1 e término em t2
    lambda_ = lambda_ * (1 - ((1 + np.tanh(t - t1)) / 2) * ((1 - np.tanh(t - t2) / 2) ) )
    
    return [-lambda_*S*I,
            lambda_*S*I - alpha*E,
            (1-p)*alpha*E - gamma*I - mu*I,
            p*alpha*E - theta*A,
            gamma*I + theta*A,
            mu*I ]

In [40]:
parametros_iniciais = [0.68,0.2,0.4,0.5,0.15,0.05,0.07,0.04,55,110] # obtidos do interact

In [41]:
problem = {
    'num_vars': 10,
    'names': ['lambda_', 'alpha', 'p', 'theta', 'gamma', 'mu', 'mu1', 'mu2', 't1', 't2'],
    'bounds': [[0.,1.],[0.,1.],[0.,1.],[0.,1.],[0.,1.],[0.,0.8],[0.,0.8],[0.,0.8],[50,60],[100,120]]
}

param_values = saltelli.sample(problem, 1000)
param_values.shape

(22000, 10)

Para fazer a análise de sensibilidade precisamos selecionar um aspecto da saída do model, sobre o qual desejamos estudar a variância em resposta à varância dos parâmetros. Para este caso simples, vamos escolher o valor de pico de $I(t)$.

In [42]:
def eval_ODE(parametros):
    
    global parametros_iniciais
    parametros_iniciais = parametros

    inits = [.99, 0,.001, 0, 0, 0]
    tspan = [0,100]
    
    T = ode_solver()
    T.function = ODE
    T.algorithm='rk8pd'
    T.ode_solve(tspan, inits, num_points=100,params = parametros)
    
    Y = max([ y[1][2] for y in T.solution])
    
    return Y

In [43]:
eval_ODE([0.68,0.2,0.4,0.5,0.15,0.05,0.07,0.04,55,110])

0.0197036972149514

In [44]:
eval_ODE(param_values[0])

/opt/conda/envs/sage/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:12: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


0.00100000000000000

In [46]:
Po = Pool()
Y = Po.map(eval_ODE, param_values)

/opt/conda/envs/sage/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:12: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/envs/sage/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:12: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/envs/sage/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:12: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/envs/sage/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:12: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/envs/sage/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:12: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/opt/conda/envs/sage/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:12: Depre

In [47]:
Po.close()

In [48]:
Si = sobol.analyze(problem, np.array(Y), print_to_console=False)

In [49]:
def plot_sobol(si,prob, order=1):
    Si_filter = {k:si[k] for k in ['ST','ST_conf','S1','S1_conf']}
    Si_df = pd.DataFrame(Si_filter, index=problem['names'][:-1])
    fig, ax = plt.subplots(1, figsize=(10,8))

    indices = Si_df[['S1','ST']]
    err = Si_df[['S1_conf','ST_conf']]

    indices.plot.bar(yerr=err.values.T,ax=ax)

plot_sobol(Si,problem)

ValueError: Shape of passed values is (10, 4), indices imply (9, 4)